<a href="https://colab.research.google.com/github/SaiSharanyabonthala/Innomatics/blob/main/final_delivery_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/order_data')

Drive already mounted at /content/order_data; to attempt to forcibly remount, call drive.mount("/content/order_data", force_remount=True).


In [ ]:
import pandas as pd
orders = pd.read_csv('/content/orders.csv')

In [ ]:
df_json = pd.read_json('/content/users.json')

In [ ]:
import sqlite3

# Create a connection
conn = sqlite3.connect(':memory:')

# Read the .sql file and execute the commands
with open('/content/restaurants.sql', 'r') as f:
    sql_script = f.read()
    conn.executescript(sql_script)

# Pull the data into  Pandas DataFrame
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

In [ ]:
import pandas as pd

merged_data = pd.merge(orders, df_json, on='user_id', how='left')


final_df = pd.merge(merged_data, restaurants, on='restaurant_id', how='left')


print(final_df.head())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3    Restaurant_224  Chinese    

In [ ]:

final_df = final_df.drop(columns=['restaurant_name_x']).rename(columns={'restaurant_name_y': 'restaurant_name'})


print(final_df.isnull().sum())


order_id           0
user_id            0
restaurant_id      0
order_date         0
total_amount       0
name               0
city               0
membership         0
restaurant_name    0
cuisine            0
rating             0
dtype: int64


In [ ]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])
final_df['month'] = final_df['order_date'].dt.month_name()

/tmp/ipython-input-2062389994.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(final_df['order_date'])


In [ ]:

pivot_table = final_df.pivot_table(index='city', columns='membership', values='total_amount', aggfunc='sum')
print(pivot_table)

membership        Gold     Regular
city                              
Bangalore    994702.59  1212243.99
Chennai     1080909.79   909603.24
Hyderabad    896740.19   992626.39
Pune        1003012.32   921785.61


In [ ]:

final_df = final_df.drop(columns=['restaurant_name']).rename(columns={'restaurant_name_y': 'restaurant_name'})


final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)


final_df['order_month'] = final_df['order_date'].dt.month_name()

final_df.to_csv('final_delivery_dataset.csv', index=False)

print("Dataset is ready for analysis!")

Dataset is ready for analysis!


In [ ]:
import pandas as pd
import json
import sqlite3


orders = pd.read_csv('orders.csv')
users = pd.read_json('users.json')


conn = sqlite3.connect(':memory:')
with open('restaurants.sql', 'r') as f:
    conn.executescript(f.read())
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()


merged_df = pd.merge(orders, users, on='user_id', how='left')

final_df = pd.merge(merged_df, restaurants, on='restaurant_id', how='left')

final_df = final_df.drop(columns=['restaurant_name_x']).rename(columns={'restaurant_name_y': 'restaurant_name'})
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)

final_df.to_csv('final_food_delivery_dataset.csv', index=False)

print("Success! 'final_food_delivery_dataset.csv' has been created.")

Success! 'final_food_delivery_dataset.csv' has been created.


In [ ]:
import pandas as pd


df = pd.read_csv('final_food_delivery_dataset.csv')
df['order_date'] = pd.to_datetime(df['order_date'])

monthly_trends = df.resample('M', on='order_date').agg({
    'order_id': 'count',      # Number of orders
    'total_amount': 'sum'     # Total cash flow
}).rename(columns={'order_id': 'Order_Count', 'total_amount': 'Revenue'})

print("--- 1. Order Trends (Monthly) ---")
print(monthly_trends)

--- 1. Order Trends (Monthly) ---
            Order_Count    Revenue
order_date                        
2023-01-31          804  645818.76
2023-02-28          785  630867.40
2023-03-31          903  716738.98
2023-04-30          812  666097.18
2023-05-31          844  668428.61
2023-06-30          784  610822.93
2023-07-31          859  688559.45
2023-08-31          851  694987.58
2023-09-30          812  653838.07
2023-10-31          863  699187.73
2023-11-30          807  637772.43
2023-12-31          849  681303.50
2024-01-31           27   17201.50


/tmp/ipython-input-1145788927.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_trends = df.resample('M', on='order_date').agg({


In [ ]:

user_behavior = df.groupby('user_id').agg({
    'order_id': 'count',
    'total_amount': 'mean'
}).rename(columns={'order_id': 'Frequency', 'total_amount': 'Avg_Ticket_Size'})

print("--- 2. User Behavior (Top 5 Active Users) ---")
print(user_behavior.sort_values(by='Frequency', ascending=False).head())

--- 2. User Behavior (Top 5 Active Users) ---
         Frequency  Avg_Ticket_Size
user_id                            
2973            13       697.159231
1515            12       963.040833
496             11       875.845455
874             11       810.595455
1337            11       809.772727


In [ ]:

city_performance = df.groupby('city')['total_amount'].sum().sort_values(ascending=False)

cuisine_quality = df.groupby('cuisine')['rating'].mean().sort_values(ascending=False)

print("--- 3a. Revenue by City ---")
print(city_performance)
print("\n--- 3b. Avg Rating by Cuisine ---")
print(cuisine_quality)

--- 3a. Revenue by City ---
city
Bangalore    2206946.58
Chennai      1990513.03
Pune         1924797.93
Hyderabad    1889366.58
Name: total_amount, dtype: float64

--- 3b. Avg Rating by Cuisine ---
cuisine
Chinese    4.099338
Italian    4.042457
Indian     4.021466
Mexican    4.020767
Name: rating, dtype: float64


In [ ]:

membership_impact = df.groupby('membership').agg({
    'total_amount': ['mean', 'sum'],
    'order_id': 'count'
})

print("--- 4. Membership Impact Analysis ---")
print(membership_impact)

--- 4. Membership Impact Analysis ---
           total_amount             order_id
                   mean         sum    count
membership                                  
Gold         797.145556  3975364.89     4987
Regular      805.158434  4036259.23     5013


In [ ]:

revenue_dist = df['total_amount'].describe()


df['month_name'] = df['order_date'].dt.month_name()
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
seasonality = df.groupby('month_name')['total_amount'].sum().reindex(month_order)

print("--- 5a. Revenue Distribution ---")
print(revenue_dist)
print("\n--- 5b. Seasonal Revenue peaks ---")
print(seasonality)

--- 5a. Revenue Distribution ---
count    10000.000000
mean       801.162412
std        405.458753
min        100.200000
25%        446.310000
50%        806.295000
75%       1149.227500
max       1499.830000
Name: total_amount, dtype: float64

--- 5b. Seasonal Revenue peaks ---
month_name
January      663020.26
February     630867.40
March        716738.98
April        666097.18
May          668428.61
June         610822.93
July         688559.45
August       694987.58
September    653838.07
October      699187.73
November     637772.43
December     681303.50
Name: total_amount, dtype: float64


In [ ]:

gold_revenue = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum()
print(gold_revenue.idxmax()) # idxmax returns the name of the highest city

Chennai


In [ ]:

cuisine_aov = df.groupby('cuisine')['total_amount'].mean()
print(cuisine_aov.idxmax())

Mexican


In [ ]:

user_spend = df.groupby('user_id')['total_amount'].sum()
high_spenders = user_spend[user_spend > 1000].count()
print(high_spenders)

2544


In [ ]:

bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0']


df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels, include_lowest=True)
print(df.groupby('rating_range')['total_amount'].sum())

rating_range
3.0-3.5    2136772.70
3.6-4.0    1717494.41
4.1-4.5    1960326.26
4.6-5.0    2197030.75
Name: total_amount, dtype: float64


/tmp/ipython-input-3728393048.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('rating_range')['total_amount'].sum())


In [ ]:
# Filter for Gold, group by city, and find average spend
gold_city_aov = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].mean()
print(gold_city_aov.idxmax())

Chennai


In [ ]:
# Aggregate unique restaurant count and total revenue
cuisine_summary = df.groupby('cuisine').agg({
    'restaurant_id': 'nunique',
    'total_amount': 'sum'
})
print(cuisine_summary.sort_values('restaurant_id'))

         restaurant_id  total_amount
cuisine                             
Chinese            120    1930504.65
Indian             126    1971412.58
Italian            126    2024203.80
Mexican            128    2085503.09


In [ ]:
# Use value_counts with normalize=True to get percentages
gold_pct = df['membership'].value_counts(normalize=True) * 100
print(round(gold_pct['Gold']))

50


In [ ]:
# Group by restaurant name, count orders and average amount
res_stats = df.groupby('restaurant_name').agg({'order_id':'count', 'total_amount':'mean'})

# Filter for low volume and sort by high spend
low_volume_high_spend = res_stats[res_stats['order_id'] < 20].sort_values('total_amount', ascending=False)
print(low_volume_high_spend.head(1))

                 order_id  total_amount
restaurant_name                        
Restaurant_294         13   1040.222308


In [ ]:
# Group by both membership and cuisine to find the top performing pair
combo_revenue = df.groupby(['membership', 'cuisine'])['total_amount'].sum()
print(combo_revenue.sort_values(ascending=False).head(1))

membership  cuisine
Regular     Mexican    1072943.3
Name: total_amount, dtype: float64


In [ ]:
# Convert date and extract the quarter (Q1, Q2, etc.)
df['order_date'] = pd.to_datetime(df['order_date'])
df['quarter'] = df['order_date'].dt.to_period('Q')

# Sum revenue by quarter
quarterly_revenue = df.groupby('quarter')['total_amount'].sum()
print(quarterly_revenue.idxmax())

2023Q3


In [ ]:
import pandas as pd

# Assuming df_final is your merged master dataset
df_final = pd.read_csv('final_food_delivery_dataset.csv')

In [ ]:
gold_orders_count = df_final[df_final['membership'] == 'Gold'].shape[0]
print(f"Total Gold Orders: {gold_orders_count}")

Total Gold Orders: 4987


In [ ]:
hyd_revenue = round(df_final[df_final['city'] == 'Hyderabad']['total_amount'].sum())
print(f"Hyderabad Total Revenue: {hyd_revenue}")

Hyderabad Total Revenue: 1889367


In [ ]:
distinct_users = df_final['user_id'].nunique()
print(f"Distinct Users with Orders: {distinct_users}")

Distinct Users with Orders: 2883


In [ ]:
gold_aov = round(df_final[df_final['membership'] == 'Gold']['total_amount'].mean(), 2)
print(f"Gold Member AOV: {gold_aov}")

Gold Member AOV: 797.15


In [ ]:
high_rating_orders = df_final[df_final['rating'] >= 4.5].shape[0]
print(f"Orders for Rating >= 4.5: {high_rating_orders}")

Orders for Rating >= 4.5: 3374


In [ ]:

gold_only = df_final[df_final['membership'] == 'Gold']
top_city = gold_only.groupby('city')['total_amount'].sum().idxmax()


order_count = gold_only[gold_only['city'] == top_city].shape[0]

print(f"Top Gold City: {top_city}")
print(f"Total Orders in {top_city}: {order_count}")

Top Gold City: Chennai
Total Orders in Chennai: 1337
